# Preguntas y Respuestas con RAG Langchain

Usando Chroma DB and LangChain para contestar preguntas de un texto, con una base de datos local. Guardamos los embaddings, y los usamos después.

In [ ]:
!pip install chromadb langchain langchain-openai

In [7]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings

import os
from dotenv import load_dotenv

In [8]:
# Cargar variables de entorno
load_dotenv()
# Configurar el motor de OpenAI
api_key=os.getenv("OPENAI_API_KEY")

## Cargar y procesar documentos.
aqui cargamos los documentos sobre los que vamosa  aplicar el Q&A

Los dividimos en trozos (chunks). ASi buscamos los más relevantes y los pasamos al LLM.

In [9]:
# Cargado y procesado del texto
loader = PyPDFLoader('empresas_emergentes.pdf')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Inicializamos PeristedChromaDB

Creamos los embeddings. Y los guardamos en la db

In [11]:
# Directorio donde se guardará la db
persist_directory = 'db'

engine = "gpt-4"
embedding = OpenAIEmbeddings(api_key=api_key, model="text-embedding-3-large")

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

## Persist the Database
In a notebook, we should call `persist()` to ensure the embeddings are written to disk.
This isn't necessary in a script - the database will be automatically persisted when the client object is destroyed.

In [12]:
vectordb.persist()
vectordb = None

## Cargamos la db del disco
`persist_directory` es el directorio donde se va a guardar la db y `embedding_function` tiene que ser la misma que cunado instanciamos a db. Se encargara de las respuestas.

In [13]:
# Cargamos la base de datos persistente del disco y usamos un langchain retrieval
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(api_key=api_key),
    retriever=vectordb.as_retriever(
        search_kwargs={"k": 5})
    )

## Preguntas y respuestas

Con langchainy RAG

In [14]:
query = "Cuál es el sentido de la vida"
result = qa.invoke(query)
result.get("result")

' Lo siento, no tengo la capacidad de responder a preguntas filosóficas o abstractas. Mi función es proporcionar información basada en el contexto proporcionado.'

In [15]:
query = "¿Qué es una empresa emergente?"
result = qa.invoke(query)
result.get("result")

' Una empresa emergente, también conocida como start-up, es una compañía joven con potencial de crecimiento y desarrollo en su sector. Suelen ser innovadoras y utilizan las nuevas tecnologías para llevar a cabo sus procesos de negocio. Tienen recursos limitados y suelen optar por métodos de financiación no convencionales. Además, su forma de trabajar es flexible y suele enfocarse en la personalización de los servicios. '

In [16]:
query = "Dame ejemplos de empresa emergente"
result = qa.invoke(query)
result.get("result")

' Algunos ejemplos de empresas emergentes son Glovo, Cabify, Airbnb, Uber, Spotify, Dropbox, Snapchat, entre otras. Todas estas empresas han introducido nuevos modelos de negocio en sus respectivos sectores y han logrado un gran éxito a nivel nacional e internacional.'

In [17]:
query = "Facturación y beneficio medio de una empresa emergente"
result = qa.invoke(query)
result.get("result")

' No se puede determinar la facturación y beneficio medio de una empresa emergente ya que varía de una empresa a otra y depende de varios factores, como el sector en el que se encuentran, la inversión inicial, la estrategia de crecimiento, entre otros.'

## Limpieza

Cuando hayas terminado con la base de datos, puedes eliminarla del disco. Puedes eliminar la colección específica con la que está trabajando (si tiene varias) o eliminar toda la base de datos destruyendo el directorio de persistencia.

In [ ]:
# limpiamos la colección
vectordb.delete_collection()
vectordb.persist()

# nos cargamos el directorio de persistencia
!rm -rf db/

Persisting DB to disk, putting it in the save folder db
